In [244]:
%load_ext autoreload
%autoreload 2

In [354]:
import re
from collections import namedtuple
from typing import List, Dict, Union, NamedTuple

import fitz
from prextract.dodf_hierarchy import is_title_subtitle
from prextract.dodf_hierarchy import are_title_subtitle
from prextract.dodf_hierarchy import drop_header_footer


In [412]:
PATH='2_jan_2019.pdf'
doc = fitz.open(PATH)
pnum = 1

p0 = doc[pnum]
p_width = p0.MediaBox[2]


class TextBlockTrans(NamedTuple):
    x0: float
    y0: float
    x1: float
    y1: float
    text: str
    block_no: int
    page: int
    pwidth: float = None
    def __repr__(self):
        ret = []
        ret.append('TextBlockTrans')
        ret.append('\tbbox: ({}, {}, {}, {})'.format(*self[:4]))
        ret.append('\ttext: {}'.format(self.text))
        ret.append('\tblock_no: {}'.format(self[5]))
        ret.append('\tpage, pwidth: {}'.format(self[-2:]))
        return '\n'.join(ret)
    

TextBlockPaged = namedtuple('TextBlockTrans',
    'x0 y0 x1 y1 text block_no page',
    defaults=7*(None,)
)

In [413]:
text_blocks = p0.getTextBlocks()
extract = p0.getTextPage().extractDICT()['blocks']
text_blocks[4:6]

[(56.69197463989258,
  368.1159362792969,
  404.076171875,
  606.9794921875,
  'AVISO DE HOMOLOGAÇÃO E CONVOCAÇÃO DE RESULTADO\nDO EDITAL 01/2018 - CONDOMÍNIO SOLAR DE BRASÍLIA.\nO Presidente da COVED, acolhendo o parecer inserido no processo abaixo, declara habilitado para a\nvenda à VISTA o item a seguir: item 817, QD 06 RUA 04 LT 01, ao interessado (a) LUZIA\nGADELHA DA SILVA GONET, pelo valor de R$ 195.299,66 (cento e noventa e cinco mil, duzentos\ne noventa e nove reais e sessenta e seis centavos), conforme proposta de compra anexada ao processo\nSEI nº 00111-00004631/2018-23. O presidente da COVED, acolhendo os pareceres inseridos nos\nprocessos abaixo, declara habilitados para a venda a PRAZO os itens a seguir: item 285, QD 02 RUA\n07 LT 27, ao interessado (a) MARISTELA DE ARAUJO CAVALCANTE, pelo valor de R$\n186.496,67 (cento e oitenta e seis mil, quatrocentos e noventa e seis reais e sessenta e sete centavos),\nconforme proposta de compra anexada ao processo SEI nº 00111-00003

In [414]:
def textBlock_topage(lis, page_width, pnum):
    """Given a text_block list (of tuples), return one with 
    two more values at the end, indicating `page number` and
    `page width`.
    """
    return [TextBlockTrans(*i[:-1], pnum, page_width) for i in lis]

def text_blocks_transform(text_blocks: List[TextBlockTrans],
                          keep_page_width=True):
    lis = []
    for idx, tb in enumerate(text_blocks):
        p_num = tb[-2]
        p_num = tb.page
        
        
        p_width = tb[-1]
        p_width = tb.pwidth
        
        
        p_num *= 2
        x0, y0, x1, y1 = tb[:4]
        x0 = tb.x0
        
        p_num += int(x0 >( p_width / 2))
        if keep_page_width:            
            lis.append( TextBlockTrans( *(tb[:-2]), p_num, p_width ) )
        else:
            lis.append( TextBlockPaged( *(tb[:-2]), p_num, ) )
    return lis

def trans_key(text_block_paged: TextBlockPaged):
    _ = text_block_paged
    return (_.page, int(_.y0), _.x0)

def sort_transformed(text_blocks_transformed: 
    List[Union[TextBlockTrans, TextBlockPaged]]):
    return sorted(text_blocks_transformed, key=trans_key)

In [415]:
tb_paged = textBlock_topage(text_blocks, p_width, pnum)
tb_paged[4:6]

[TextBlockTrans
 	bbox: (56.69197463989258, 368.1159362792969, 404.076171875, 606.9794921875)
 	text: AVISO DE HOMOLOGAÇÃO E CONVOCAÇÃO DE RESULTADO
 DO EDITAL 01/2018 - CONDOMÍNIO SOLAR DE BRASÍLIA.
 O Presidente da COVED, acolhendo o parecer inserido no processo abaixo, declara habilitado para a
 venda à VISTA o item a seguir: item 817, QD 06 RUA 04 LT 01, ao interessado (a) LUZIA
 GADELHA DA SILVA GONET, pelo valor de R$ 195.299,66 (cento e noventa e cinco mil, duzentos
 e noventa e nove reais e sessenta e seis centavos), conforme proposta de compra anexada ao processo
 SEI nº 00111-00004631/2018-23. O presidente da COVED, acolhendo os pareceres inseridos nos
 processos abaixo, declara habilitados para a venda a PRAZO os itens a seguir: item 285, QD 02 RUA
 07 LT 27, ao interessado (a) MARISTELA DE ARAUJO CAVALCANTE, pelo valor de R$
 186.496,67 (cento e oitenta e seis mil, quatrocentos e noventa e seis reais e sessenta e sete centavos),
 conforme proposta de compra anexada ao proce

In [416]:
print(tb_paged[0])
tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
tb_trans[5:7]
# [i.text for i in drop_header_footer(sort_transformed(
#     tb_trans
# ))]

TextBlockTrans
	bbox: (56.66706848144531, 55.11705017089844, 765.406005859375, 70.56302642822266)
	text: PÁGINA 2
Nº 1, quarta-feira, 2 de janeiro de 2019
Diário Oficial do Distrito Federal
	block_no: 0
	page, pwidth: (1, 814.9609985351562)


[TextBlockTrans(x0=56.691986083984375, y0=614.0555419921875, x1=404.0885925292969, y1=718.0698852539062, text='AVISO DE HOMOLOGAÇÃO E CONVOCAÇÃO DE RESULTADO\nDO EDITAL 01/2017 - VICENTE PIRES- ETAPA-3.\nO Presidente da COVED, acolhendo os pareceres inseridos nos processos abaixo, declara habilitados\npara a venda à VISTA os itens a seguir: item 18, SHVP TRECHO 03 QD EPTG 01 CONJ 02 LT\n19, ao interessado (a) OLGA RODRIGUES DE SOUZA BARBOSA, pelo valor de R$ 135.091,00\n(cento e trinta e cinco mil e noventa e um reais), conforme proposta de compra anexada ao processo\nSEI nº 00111-00017876/2017-30; item 372, SHVP TRECHO 03 QD 01 CONJ 07 LT 06, ao\ninteressado (a) VILMA DOMINGUES DE SOUZA , pelo valor de R$ 37.371,32 ( trinta e sete mil,\ntrezentos e setenta e um reais e trinta e dois centavos ), conforme proposta de compra anexada ao\nprocesso SEI nº 00111-00016512/2017-32 ; item 377 , SHVP TRECHO 03 QD 01 CONJ 07 LT 15\n, ao interessado(a) MARIA NEIDE DE SOUZA , pelo valor de R$ 34.79

In [417]:
# tb_trans[50]._asdict()

## VERIFICAR SE UM BLOCO É CANDIDATO A TÍTULO

In [418]:
import re
UPPER_REG = re.compile(r'[A-Z]')
def is_upper(text: str):
    return text.upper() == text

In [419]:
def get_block_spans(block): 
    span_lis = [] 
    for line in block['lines']: 
        for span in line['spans']: 
            span_lis.append(span) 
    return span_lis

titles_idx = [idx for (idx, bl) in enumerate(tb_trans)
          if is_upper(bl.text)]


In [420]:
block_spans = {}
for i in titles_idx:
    block_spans[i] = get_block_spans(extract[i])

In [421]:
block_spans.keys()

dict_keys([2, 8, 9, 10, 11, 12, 13, 14, 15])

In [422]:
for k, span_list in block_spans.items():
    if all(are_title_subtitle(span_list)):
        print('\n'.join((sp['text'] for sp in span_list)))
        print('\t----- {} -----\n'.format(k))

SECRETARIA DE ESTADO DE ECONOMIA,
	----- 9 -----

DESENVOLVIMENTO, INOVAÇÃO, CIÊNCIA E
	----- 10 -----

TECNOLOGIA
	----- 11 -----

SECRETARIA DE ESTADO DE ECONOMIA,
	----- 12 -----

DESENVOLVIMENTO, INOVAÇÃO, CIÊNCIA E
	----- 13 -----

TECNOLOGIA
	----- 14 -----

COMPANHIA IMOBILIÁRIA DE BRASÍLIA
	----- 15 -----



In [423]:
def mount_hierarchy(page: fitz.Page):
#     TODO : fazer para o documento inteiro
    p_width = page.MediaBox[2]
    pnum = 0
    
    text_blocks = page.getTextBlocks()
    extract = page.getTextPage().extractDICT()['blocks']

    tb_paged = textBlock_topage(text_blocks, p_width, pnum)    
    tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
    
#     titles_idx = [idx for (idx, bl) in enumerate(tb_trans)
#           if is_upper(bl.text)]
#     block_spans = {}
#     for i in range(len(extract)):
#         block_spans[i] = get_block_spans(extract[i])
#     return [i.page for i in tb_trans]
    
    cleaned_and_sorted = drop_header_footer(sort_transformed(tb_trans))
#     return cleaned_and_sorted

    hier = [('preambulo', [])]
    
    last_title = 'preambulo'

    for text_block in cleaned_and_sorted:
        spans = get_block_spans(extract[text_block.block_no])
        if all(are_title_subtitle(spans)):
            last_title = '\n'.join([sp['text'] for sp in spans])
            hier.append((last_title, []))
        else:
            hier[-1][1].append(text_block.text)
    return hier
            
            

In [424]:
h=mount_hierarchy(p0)

In [425]:
h[0]

('preambulo', ['SEÇÃO III'])

In [426]:
h

[('preambulo', ['SEÇÃO III']),
 ('SECRETARIA DE ESTADO DE ECONOMIA,', []),
 ('SECRETARIA DE ESTADO DE ECONOMIA,', []),
 ('DESENVOLVIMENTO, INOVAÇÃO, CIÊNCIA E', []),
 ('DESENVOLVIMENTO, INOVAÇÃO, CIÊNCIA E', []),
 ('TECNOLOGIA', []),
 ('TECNOLOGIA', []),
 ('COMPANHIA IMOBILIÁRIA DE BRASÍLIA',
  ['AGÊNCIA DE DESENVOLVIMENTO DO DISTRITO FEDERAL\nCOMISSÃO DE VENDA DIRETA',
   'AVISO DE HOMOLOGAÇÃO E CONVOCAÇÃO DE RESULTADO\nDO EDITAL 01/2017 - CONDOMÍNIO JARDIM BOTÂNICO\nO Presidente da COVED, acolhendo os pareceres inseridos nos processos abaixo, declara habilitados\npara a venda à PRAZO os itens a seguir: item 80 - A , ESTÂNCIA JARDIM BOTÂNICO , SHJB\nQD 03 Rua 03 LT 112 , ao interessado(a) VANIA RUAS DE MORAIS COSTA , pelo valor de R$\n355.410,00 ( trezentos e cinquenta e cinco mil, quatrocentos e dez reais ), conforme proposta de\ncompra anexada ao processo SEI nº 00111-00020866/2017-81 ; item 171 - A , ESTÂNCIA JARDIM\nBOTÂNICO , SHJB QD 03 Rua 05 LT 144 , ao interessado(a) FRANCISCO